In [1]:
from nir.tokenizers import Regex, BioCleanTokenizer, BioCleanTokenizer2, Regex2
from utils import collection_iterator_fn

import os
import json
import time

cache_folder = "/backup/NIR_BioASQ"

index_name = "bioasq_9b"

zipped_collection = "/backup/pubmed_archive_json/"+index_name+".tar.gz"

In [2]:
#_class = Regex
_class = Regex2
#_class = BioCleanTokenizer
#_class = BioCleanTokenizer2

tk = _class.load_from_json(cache_folder=os.path.join(cache_folder, "tokenizers"), 
                           prefix_name=index_name)

print(_class.__name__)

DEBUG created tokenizer bioasq_9b_Regex2Tokenizer
False False
Regex2


In [3]:
min_freq=0

tk.update_min_word_frequency(min_freq)

In [4]:
tk.vocabulary_size()

6232199

In [5]:
from gensim.models.callbacks import CallbackAny2Vec

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

emb_size = 200
_iter=15
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1
        if self.epoch == 5:
            model.wv.save(fname_or_handle ="/backup/pre-trained_embeddings/word2vec/"+index_name+"_gensim_iter_"+str(5)+"_freq"+str(min_freq)+"_"+str(emb_size)+"_"+_class.__name__+"_word2vec.bin")
        elif self.epoch == 10:
            model.wv.save(fname_or_handle ="/backup/pre-trained_embeddings/word2vec/"+index_name+"_gensim_iter_"+str(10)+"_freq"+str(min_freq)+"_"+str(emb_size)+"_"+_class.__name__+"_word2vec.bin")
        elif self.epoch == 15:
            model.wv.save(fname_or_handle ="/backup/pre-trained_embeddings/word2vec/"+index_name+"_gensim_iter_"+str(_iter)+"_freq"+str(min_freq)+"_"+str(emb_size)+"_"+_class.__name__+"_word2vec.bin")

In [6]:
def sentences_generator():
    
    corpus_generator = collection_iterator_fn(zipped_collection, f_map=lambda x:x["title"]+" "+x["abstract"])
    
    with open("training9b.json", "r") as f:
        queries = list(map(lambda x:x["body"], json.load(f)["questions"]))
        queries = tk.texts_to_sequences(queries)
                
    print("word2vec queries")
    for q in queries:
        yield list(map(lambda x: tk.index_word[x], q))

    print("word2vec docs")
    for subarticles in corpus_generator():
        s_time = time.time()
        subarticles = tk.texts_to_sequences(subarticles)
        print("tokenizer time", time.time() - s_time)
        for article in subarticles:
            yield list(map(lambda x: tk.index_word[x], article))

In [7]:
corpus = [x for x in sentences_generator()]

[CORPORA] Openning tar file /backup/pubmed_archive_json/bioasq_9b.tar.gz
word2vec queries
word2vec docs
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_00000000_to_03000000
tokenizer time 770.4341924190521
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_03000000_to_06000000
tokenizer time 753.0616497993469
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_06000000_to_09000000
tokenizer time 726.5105102062225
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_09000000_to_12000000
tokenizer time 702.253538608551
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_12000000_to_15000000
tokenizer time 696.132253408432
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_15000000_to_18000000
tokenizer time 698.4852187633514
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_18000000_to_21000000
tokenizer time 695.2347025871277
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_21000000_to_21165909
tokenizer time 38.567466020584106


In [ ]:
model = Word2Vec(corpus, size=emb_size, iter=_iter, window=5, min_count=0, workers=12, sg=1, negative=5, callbacks=[EpochLogger()])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
